# Double pendulum demonstration

For an explanation, visit [Eric Weisstein's World of Physics: Double Pendulum](https://scienceworld.wolfram.com/physics/DoublePendulum.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import matplotlib.animation as animation
from IPython.display import HTML

def animate_double_pendulum_side_by_side(L1, L2, m1, m2, omega1_init1, omega2_init1, omega1_init2, omega2_init2, g=9.81):
    """
    Animate two double pendulums side by side with slightly different initial angular velocities.

    Parameters:
    - L1: Length of the first rod
    - L2: Length of the second rod
    - m1: Mass of the first pendulum
    - m2: Mass of the second pendulum
    - omega1_init1: Initial angular velocity for the first pendulum (plot 1)
    - omega2_init1: Initial angular velocity for the second pendulum (plot 1)
    - omega1_init2: Initial angular velocity for the first pendulum (plot 2)
    - omega2_init2: Initial angular velocity for the second pendulum (plot 2)
    - g: Acceleration due to gravity (default is 9.81 m/s^2)
    """

    def double_pendulum_derivatives(state, t):
        theta1, omega1, theta2, omega2 = state
        delta = theta2 - theta1
        
        # Equations of motion
        denominator1 = (m1 + m2) * L1 - m2 * L1 * np.cos(delta) ** 2

        dtheta1_dt = omega1
        dtheta2_dt = omega2
        
        domega1_dt = (m2 * L1 * omega1 ** 2 * np.sin(delta) * np.cos(delta) +
                      m2 * g * np.sin(theta2) * np.cos(delta) +
                      m2 * L2 * omega2 ** 2 * np.sin(delta) -
                      (m1 + m2) * g * np.sin(theta1)) / denominator1
        
        domega2_dt = (-L2 / L1) * domega1_dt * np.cos(delta) + g * np.sin(theta1) / L1
        
        return [dtheta1_dt, domega1_dt, dtheta2_dt, domega2_dt]

    # Initial conditions for the two pendulums
    initial_state1 = [np.pi / 2, omega1_init1, np.pi / 2, omega2_init1]
    initial_state2 = [np.pi / 2, omega1_init2, np.pi / 2, omega2_init2]

    # Time points
    t = np.linspace(0, 20, 200)

    # Solve the differential equations for both pendulums
    trajectory1 = odeint(double_pendulum_derivatives, initial_state1, t)
    trajectory2 = odeint(double_pendulum_derivatives, initial_state2, t)

    # Extract angles for both trajectories
    theta1_1, omega1_1, theta2_1, omega2_1 = trajectory1.T
    theta1_2, omega1_2, theta2_2, omega2_2 = trajectory2.T

    # Calculate x and y positions of the pendulum masses for both trajectories
    x1_1 = L1 * np.sin(theta1_1)
    y1_1 = -L1 * np.cos(theta1_1)
    x2_1 = x1_1 + L2 * np.sin(theta2_1)
    y2_1 = y1_1 - L2 * np.cos(theta2_1)

    x1_2 = L1 * np.sin(theta1_2)
    y1_2 = -L1 * np.cos(theta1_2)
    x2_2 = x1_2 + L2 * np.sin(theta2_2)
    y2_2 = y1_2 - L2 * np.cos(theta2_2)

    # Set up the figure and axes for side-by-side plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.set_xlim(-2, 2)
    ax1.set_ylim(-2, 2)
    ax1.set_title(f'Pendulum 1 (ω1={omega1_init1}, ω2={omega2_init1})')

    ax2.set_xlim(-2, 2)
    ax2.set_ylim(-2, 2)
    ax2.set_title(f'Pendulum 2 (ω1={omega1_init2}, ω2={omega2_init2})')

    # Pendulum lines, mass markers, and track for both plots
    line1, = ax1.plot([], [], 'o-', lw=2)
    mass1_1, = ax1.plot([], [], 'o', color='blue')
    mass2_1, = ax1.plot([], [], 'o', color='red')
    track1, = ax1.plot([], [], 'g-', lw=0.5)

    line2, = ax2.plot([], [], 'o-', lw=2)
    mass1_2, = ax2.plot([], [], 'o', color='blue')
    mass2_2, = ax2.plot([], [], 'o', color='red')
    track2, = ax2.plot([], [], 'g-', lw=0.5)

    # Track storage for the pendulum masses
    x2_track1 = []
    y2_track1 = []
    x2_track2 = []
    y2_track2 = []

    def init():
        line1.set_data([], [])
        mass1_1.set_data([], [])
        mass2_1.set_data([], [])
        track1.set_data([], [])

        line2.set_data([], [])
        mass1_2.set_data([], [])
        mass2_2.set_data([], [])
        track2.set_data([], [])
        return line1, mass1_1, mass2_1, track1, line2, mass1_2, mass2_2, track2

    def update(i):
        # Update the first pendulum
        line1.set_data([0, x1_1[i], x2_1[i]], [0, y1_1[i], y2_1[i]])
        mass1_1.set_data([x1_1[i]], [y1_1[i]])
        mass2_1.set_data([x2_1[i]], [y2_1[i]])
        # Append the current position to the track
        x2_track1.append(x2_1[i])
        y2_track1.append(y2_1[i])
        track1.set_data(x2_track1, y2_track1)

        # Update the second pendulum
        line2.set_data([0, x1_2[i], x2_2[i]], [0, y1_2[i], y2_2[i]])
        mass1_2.set_data([x1_2[i]], [y1_2[i]])
        mass2_2.set_data([x2_2[i]], [y2_2[i]])
        # Append the current position to the track
        x2_track2.append(x2_2[i])
        y2_track2.append(y2_2[i])
        track2.set_data(x2_track2, y2_track2)

        return line1, mass1_1, mass2_1, track1, line2, mass1_2, mass2_2, track2

    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=len(t), init_func=init, blit=True, interval=50)

    # Display the animation in the notebook
    return HTML(ani.to_jshtml())

In [ ]:
# Example usage: Slightly different initial angular velocities
animate_double_pendulum_side_by_side(L1=1.0, L2=1.0, m1=1.0, m2=1.0, 
                                      omega1_init1=0, omega2_init1=0, 
                                      omega1_init2=0.1, omega2_init2=0.1)